In [3]:
import pandas as pd
from pandas_profiling import ProfileReport

df = pd.read_csv('Hate_Crimes_by_County_and_Bias_Type__Beginning_2010.csv')

First, see which values are in the unit column:

In [6]:
df['Year'].value_counts()

2017    72
2010    68
2013    66
2011    62
2016    61
2012    59
2019    57
2014    56
2018    53
2015    51
Name: Year, dtype: int64

Next, filter the dataframe to only keep the 'years' values:

In [3]:
years_df = df[df['COMMITMENT_UNIT'] == 'Year(s)'].copy()

Convert the term (now only years) to numeric, with any errors turned into missing values:

In [4]:
# coerce turns errors into NaN (missing values)
years_df['COMMITMENT_TERM'] = pd.to_numeric(years_df['COMMITMENT_TERM'], errors='coerce')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 605 entries, 0 to 604
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   County                                        605 non-null    object
 1   Year                                          605 non-null    int64 
 2   Crime Type                                    605 non-null    object
 3   Anti-Male                                     605 non-null    int64 
 4   Anti-Female                                   605 non-null    int64 
 5   Anti-Transgender                              605 non-null    int64 
 6   Anti-Gender Identity Expression               605 non-null    int64 
 7   Anti-Age*                                     605 non-null    int64 
 8   Anti-White                                    605 non-null    int64 
 9   Anti-Black                                    605 non-null    int64 
 10  An

An easy way to select columns is to copy-paste the list of columns into a new cell, and delete columns we don't want. Here, we are keeping columns that look like they have data we want to investigate and don't have lots of missing values:

In [8]:
df.columns

Index(['County', 'Year', 'Crime Type', 'Anti-Male', 'Anti-Female',
       'Anti-Transgender', 'Anti-Gender Identity Expression', 'Anti-Age*',
       'Anti-White', 'Anti-Black', 'Anti-American Indian/Alaskan Native',
       'Anti-Asian', 'Anti-Native Hawaiian/Pacific Islander',
       'Anti-Multi-Racial Groups', 'Anti-Other Race', 'Anti-Jewish',
       'Anti-Catholic', 'Anti-Protestant', 'Anti-Islamic (Muslim)',
       'Anti-Multi-Religious Groups', 'Anti-Atheism/Agnosticism',
       'Anti-Religious Practice Generally', 'Anti-Other Religion',
       'Anti-Buddhist', 'Anti-Eastern Orthodox (Greek, Russian, etc.)',
       'Anti-Hindu', 'Anti-Jehovahs Witness', 'Anti-Mormon',
       'Anti-Other Christian', 'Anti-Sikh', 'Anti-Hispanic', 'Anti-Arab',
       'Anti-Other Ethnicity/National Origin', 'Anti-Non-Hispanic*',
       'Anti-Gay Male', 'Anti-Gay Female', 'Anti-Gay (Male and Female)',
       'Anti-Heterosexual', 'Anti-Bisexual', 'Anti-Physical Disability',
       'Anti-Mental Disability

In [17]:
# keep only some columns
keep_cols = ['OFFENSE_CATEGORY','DISPOSITION_DATE',
       'DISPOSITION_CHARGED_OFFENSE_TITLE', 'CHARGE_COUNT', 'CHARGE_DISPOSITION',
       'SENTENCE_PHASE',
       'SENTENCE_TYPE',
       'COMMITMENT_TERM',
       'RACE', 'GENDER',
       ]
keep_df = years_df[keep_cols].copy()

In [18]:
keep_df.head()

,OFFENSE_CATEGORY,DISPOSITION_DATE,DISPOSITION_CHARGED_OFFENSE_TITLE,CHARGE_COUNT,CHARGE_DISPOSITION,SENTENCE_PHASE,SENTENCE_TYPE,COMMITMENT_TERM,RACE,GENDER
1,PROMIS Conversion,12/17/2014 12:00:00 AM,HOME INVASION,14,Nolle On Remand,Original Sentencing,Conversion,30.0,Black,Male
4,PROMIS Conversion,12/17/2014 12:00:00 AM,HOME INVASION,13,Plea Of Guilty,Amended/Corrected Sentencing,Prison,30.0,Black,Male
5,PROMIS Conversion,12/17/2014 12:00:00 AM,HOME INVASION,13,Plea Of Guilty,Original Sentencing,Conversion,30.0,Black,Male
6,PROMIS Conversion,12/17/2014 12:00:00 AM,ARMED ROBBERY,15,Plea Of Guilty,Amended/Corrected Sentencing,Prison,30.0,Black,Male
7,PROMIS Conversion,12/17/2014 12:00:00 AM,ARMED ROBBERY,15,Plea Of Guilty,Original Sentencing,Conversion,30.0,Black,Male


We convert this column to a datetime type, which allows us to do things like `keep_df['DISPOSITION_DATE'].dt.year` and other datetime methods. We could also use arguments like `parse_datetimes` and `infer_datetime_format` in `read_csv` to do this.

In [19]:
keep_df['DISPOSITION_DATE'] = pd.to_datetime(keep_df['DISPOSITION_DATE'], errors='coerce')

In [20]:
keep_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178836 entries, 1 to 248141
Data columns (total 10 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   OFFENSE_CATEGORY                   178836 non-null  object        
 1   DISPOSITION_DATE                   178832 non-null  datetime64[ns]
 2   DISPOSITION_CHARGED_OFFENSE_TITLE  178836 non-null  object        
 3   CHARGE_COUNT                       178836 non-null  int64         
 4   CHARGE_DISPOSITION                 178836 non-null  object        
 5   SENTENCE_PHASE                     178836 non-null  object        
 6   SENTENCE_TYPE                      178836 non-null  object        
 7   COMMITMENT_TERM                    178834 non-null  float64       
 8   RACE                               178174 non-null  object        
 9   GENDER                             178432 non-null  object        
dtypes: datetime64[ns](1)

In [21]:
keep_df.shape

(178836, 10)

In [12]:
# drop rows with any missing values - we lost several rows but not too many
df.dropna(inplace=True)
df.shape

(605, 44)

In [10]:
# EDA 

df.plot.scatter(x = 'GENDER', y = 'RACE' title = 'Gender')

SyntaxError: invalid syntax (<ipython-input-10-acfbdbe8d549>, line 3)

In [11]:
# save the auto-EDA report as sentencing_eda.html
report = ProfileReport(df)
report.to_file('eda.html')

Summarize dataset:   0%|          | 0/57 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]